In [3]:
# Generator vs Discriminator
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [4]:
# Hyperparameters
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28 * 28
n_noise = 128

In [5]:
# GAN is unsupervised learning
# Discriminator gets real image, fake image
# Generator gets noise
# Z is noise
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

In [7]:
# W1 for hidden layer
# W2 for output layer. Same with image. 28 * 28.
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

In [8]:
# 실제 이미지를 판별하는 구부자 신경망과 생성한 이미지를 판별하는 구분자 신경망은 같은 변수를 사용해야 함.
# 같은 신겨망으로 구분을 시켜야 진짜 이미지와 가짜 이미지를 구분하는 특징들을 동시에 잡아낼 수 있음

# 은닉층을 생성자와 동일하게 구성.
# 구분자는 진짜와 얼마나 가까운가를 판단하는 값으로, 0 ~ 1사이의 값을 가짐.
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [9]:
# Generator network
def generator(noise_z):
    hidden = tf.nn.relu(
            tf.matmul(noize_z, G_W1) + G_b1)
    
    output = tf.nn.sigmoid(
            tf.matmul(hidden, G_W2) + G_b2)
    
    return output

In [15]:
# Generator network
def generator(noise_z):
    hidden = tf.nn.relu(
            tf.matmul(noise_z, G_W1) + G_b1)
    
    output = tf.nn.sigmoid(
            tf.matmul(hidden, G_W2) + G_b2)
    
    return output

# 생성자는 무작위로 생성한 노이즈를 받아 가중치와 편향을 반영하여 은닉층을 만들고,
# 은닉층에서 실제 이미지와 같은 크기의 결과값을 출력하는 구성

In [16]:
def discriminator(inputs):
    hidden = tf.nn.relu(
            tf.matmul(inputs, D_W1) + D_b1)
    
    output = tf.nn.sigmoid(
            tf.matmul(hidden, D_W2) + D_b2)
    
    return output
# 구분자는 0~1의 스칼라 값을 출력.

In [17]:
# 무작위한 노이즈를 만들어주는 함수
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [18]:
# 노이즈 Z를 이용해 가짜이미지를 만들 생성자 G를 만들고
# 이 G가 만든 가짜 이미지와 진짜 이미지 X를 구분자에 넣어 입력한 이미지가 진짜인지 판별하게 함
G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

In [19]:
# 손실값은 두개가 필요
# 생성자가 만든 이미지를 구분자가 가짜라고 판단하는 손실값 (경찰 학습용)
# 진짜라고 판단하도록 하는 손실값 (위조지폐범 학습용)
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))

In [20]:
# 위조지폐범 학습은 가짜 이미지 판별값 D_gene를 1에 가깝게 만들기만 하면 됨
# 즉 가짜이미지를 넣어도 진짜 같다고 판별해야 함
# loss_G = tf.reduce_mean(tf.log(D_gene))

In [21]:
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [23]:
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D,
                                                        var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G,
                                                        var_list=G_var_list)

In [24]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

In [ ]:
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D],
                                 feed_dict = {X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                 feed_dict = {Z: noise})
        
    print('Epoch:', '%04d' % epoch,
        'D loss: {:.4}'.format(loss_val_D),
        'G loss: {:.4}'.format(loss_val_G))

In [26]:
if epoch == 0 or (epoch + 1) % 10 == 0:
    sample_size = 10
    noise = get_noise(sample_size, n_noise)
    samples = sess.run(G, feed_dict = {Z: noise})
    
    fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
    
    for i in range(sample_size):
        ax[i].set_axis_off()
        ax[i].imshow(np.reshape(samples[i], (28, 28)))
        
    plt.savefig('samples/{}.png'.format(str(epoch)))